In [ ]:
import os
import json
import math
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist

In [ ]:
source_path = '/Users/aravdhoot/Remote-PD-Detection/frames'
destination_path = '/Users/aravdhoot/Remote-PD-Detection/skeleton_15'
json_path = '/Users/aravdhoot/Remote-PD-Detection/JSON'

In [ ]:
def calculate_distance(coord1, coord2):
    return math.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)

In [ ]:
def create_nearest_neighbors_dict(coords):
    nearest_neighbors_dict = {}

    for coord1 in coords:
        min_distance = float('inf')
        nearest_neighbor = None

        for coord2 in coords:
            if coord1 != coord2:
                distance = calculate_distance(coord1, coord2)
                if distance < min_distance and coord2 not in nearest_neighbors_dict.values():
                    min_distance = distance
                    nearest_neighbor = coord2

        nearest_neighbors_dict[coord1] = nearest_neighbor

    return nearest_neighbors_dict

In [ ]:
def place_keypoints(json_location, image_location, destination_location, frame, last_n):

    with open(json_location, 'r') as file:
        data = json.load(file)

    x_coordinate_list = list()
    y_coordinate_list = list()
    temp_dict = dict()
    temp_list = list()

    try:
        keypoints_list = data[frame]['keypoints']
        for i in range(0, len(keypoints_list), 3):
            x = keypoints_list[i]
            y = keypoints_list[i + 1]
            temp_dict[keypoints_list[i + 2]] = (x, y)

        temp_dict = dict(sorted(temp_dict.items()))
        temp_list = list(temp_dict.values())[-last_n:]

        x_coordinate_list = [value[0] for value in temp_list]
        y_coordinate_list = [value[1] for value in temp_list]
        coordinate_list = list(zip(x_coordinate_list, y_coordinate_list))

        data = plt.imread(image_location)
        plt.imshow(data)
        
        results = create_nearest_neighbors_dict(coordinate_list)

        for key, value in results.items():
            plt.scatter(key[0], key[1], color='red')
            plt.scatter(value[0], value[1], color='red')
            plt.plot([key[0], value[0]], [key[1], value[1]], color='green')

        plt.axis('off')
        plt.savefig(destination_location)
        plt.show()
        plt.close()
    except: pass

In [ ]:
os.makedirs(destination_path, exist_ok=True)
for severity in os.listdir(os.path.join(source_path)):
    if not os.path.isdir(os.path.join(source_path, severity)): os.remove(os.path.join(source_path, severity))
    os.makedirs(os.path.join(destination_path, severity), exist_ok=True)
    for video in os.listdir(os.path.join(source_path, severity)):
        if not os.path.isdir(os.path.join(source_path, severity, video)): os.remove(os.path.join(source_path, severity, video))
        os.makedirs(os.path.join(destination_path, severity, video), exist_ok=True)
        for image in os.listdir(os.path.join(source_path, severity, video)): 
            json_location = os.path.join(json_path, severity, video.split('.')[0] + '.json')
            image_location = os.path.join(source_path, severity, video, image)
            destination_location = os.path.join(destination_path, severity, video, image)
            frame = int(image.split('.')[0])
            place_keypoints(json_location, image_location, destination_location, frame, 15)